# 1 n*m 크기의 임의의 오분류 테스트 데이터의 피처 맵 K개 생성
* random.seed(42)로 랜덤하게 나오는 것을 일정하게 만들어준다
* 피처 맵 계산 연습을 위한 K개의 n*m 사이즈의 테스트 데이터셋 생성

In [ ]:
import numpy as np
import random

random_seed_id = 42

random.seed(random_seed_id)
random_start = 1; random_end = 10000

K = 1000
n = 5; m = 6

# shape이 (1, n, m)인 빈 np.array를 만듦
WTD_set = np.empty((K, n, m), dtype="int32")

for i in range(K):
    WTD = np.array([[random.randint(random_start, random_end) for i in range(m)] for j in range(n)], dtype="int32")
    # np.append는 내장 append 함수와 다르게 차원이 같아야 append됨
    WTD_set = np.append(WTD_set, WTD.reshape(1, n, m), axis = 0)

WTD_set = np.delete(WTD_set, 0, axis=0)

# print(WTD_set)

# 2 베이스 피처맵 임의로 할당

실제로는 학습 데이터의 피처 맵의 평균인 베이스 피처 맵을 입력으로 받아야 함.

In [ ]:
# 랜덤하게 나오는 값 고정
random.seed(random_seed_id)

BFM = np.array([[random.randint(random_start, random_end) for i in range(m)] for j in range(n)], dtype="int32")

# print(BFM)

# 3 각 인덱스의 유클리드 거리에 대한 중간값, 평균, 표준편차 행렬을 구함
* 각 인덱스 (i,j)에서 오분류 테스트 데이터의 피처 맵과 베이스 피처 맵 사이의 유클리드 거리를 K개 나열할 때 이에 대한 중간값, 평균, 표준편차를 저장하는 행렬들 생성

**중간값**과 **평균**과 **표준편차**도 저장함

In [ ]:
# 각 인덱스 (i,j)에서 middle의 원소들은 그 인덱스 (i,j)에서 베이스 피처 맵와 오분류 테스트 데이터의 피처 맵의 거리를 K개 나열할 때
# 이 값들에 대한 중간값이어야 한다.
middle = np.empty((n,m), dtype="int32")
mean = np.empty((n,m), dtype="int32")
std = np.empty((n,m), dtype="int32")
arrays_index = np.empty((n,m)).tolist()

for i in range(n):
    for j in range(m):
        array_i_j = np.empty((0), dtype="int32")
        for k in range(K):
            # 거리를 구해야 하므로 내장함수 abs를 씀
            array_i_j = np.append(array_i_j, abs(BFM[i][j] - WTD_set[k][i][j]))

        arrays_index[i][j] = array_i_j
        middle[i][j] = sorted(array_i_j)[K // 2]
        mean[i][j] = array_i_j.mean()
        std[i][j] = array_i_j.std()

# print("middle")
# print(middle)
# print("mean")
# print(mean)
# print("middle > mean")
# print(middle > mean)
# print("std")
# print(std)

# 4 베이스 피처 맵과 테스트 데이터의 피처맵 사이의 거리를 구함.



### 4.1 처음 착수보고서에 작성된 거리를 구하는 알고리즘은 다음과 같음

In [ ]:
WTD_set_FMD = np.empty((0), dtype="int32")
for k in range(K):
    WTD_k_FMD = 0
    for i in range(n):
        for j in range(m):
            WTD_k_FMD += (abs(BFM[i][j] - WTD_set[k][i][j]) / middle[i][j])**2
    
    WTD_set_FMD = np.append(WTD_set_FMD, WTD_k_FMD)

# for k in range(K):
#     print(WTD_set_FMD[k])

# print(WTD_set_FMD)

[ 0.         34.69341068 40.58354358 70.16734378 42.92981429 53.92175543
 54.67786371 56.68905564 50.87158193 68.90752856 57.06366165 36.66982808
 44.76153446 46.82242554 58.26472024 46.83790723 48.21265978 43.48241585
 42.46380338 30.38552967 33.51198509 45.4978096  52.2973586  53.71872721
 46.50234943 35.08504001 61.72758895 60.40455787 55.5195223  73.8997198
 43.11272176 57.79964415 54.05764376 57.9208292  50.42620781 48.55650478
 52.49874192 57.01758403 33.53156384 52.43645768 37.66263022 54.53378521
 48.20472605 50.04415263 46.94850175 43.8882461  33.44672104 46.12568397
 53.95637933 49.66105719 49.26613418 60.57151436 47.42053648 39.60968565
 45.55744284 47.12706967 48.61853399 69.409079   44.91215302 45.92775775
 46.20910991 45.05428197 38.33929389 39.09709927 54.38325196 46.30516436
 38.58781831 53.67060139 45.34485323 44.15321785 66.92291704 59.61600785
 49.76074631 50.37059913 50.27011411 42.75711824 43.02272721 59.85289123
 59.602405   47.23230972 51.43839172 58.08649703 41.

## 4.2 표준화를 시켜서 거리를 구함

### 4.2.1 표준화 시켜서 구하기 전에 데이터에 대한 특징을 좀 알아봄

# *연습

## 1. 각 인덱스에서 K개의 오분류 데이터에 대한 평균과 중간값의 관계를 살펴봄

In [ ]:
from matplotlib import pyplot as plt

# 각 인덱스에서 평균보다 낮은 값들의 수와 높은 값들의 수를 출력
for i in range(n):
    for j in range(m):
        higher_than_mean = np.array((arrays_index[i][j] > mean[i][j]))
        higher_count = np.count_nonzero(higher_than_mean)
        print(f"(L: {K - higher_count}, H: {higher_count})", end = ' ')
    print()

normal = np.empty((0), dtype="int32")

for i in range(1,2):
    for j in range(1,2):
        for k in range(K):
            normal = np.append(normal, (arrays_index[i][j][k] - mean[i][j]) / std[i][j])

X = [i for i in range(len(normal))]

plt.plot(X, normal)

plt.show()

# normal.sort()
# print(normal)

# print(len(normal[normal < 0]))

## 2. 행렬 크기에 상관없이 거리 계산을 적용할 수 있도록 만들기

이건 아마도 필요 없을 것 같다.

In [ ]:
# 오분류 테스트 데이터의 데이터 모양 구하기
data = WTD_set[0]
shape = data.shape
# 데이터 모양의 각 차원에 대한 index_max 구해서 N에 넣기
N = np.empty((0), dtype="int32")
print(N)
for dimension_index_max in shape:
    N = np.append(N, dimension_index_max)

N = np.array(N)

print(N)

I = [0 for i in range(len(N))]

middle_ = np.empty(N, dtype="int32")
mean_ = np.empty(N, dtype="int32")
std_ = np.empty(N, dtype="int32")
arrays_index_ = np.empty(N).tolist()

def set_matrices(I, D):
    if D == -1:
        for k in K:
            pass

        return

    for i in range(N[D]):
        I[D] = i
        set_matrices(I, D-1)

#

[]
[5 6]


# 5 resnet 모델 가지고 오기

In [ ]:
from torchvision import models
from torchsummary import summary
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 학습 환경 설정
model = models.resnet50(pretrained=True).to(device) # true 옵션으로 사전 학습된 모델을 로드


# summary(model, (3, 128, 128))